In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import sys 
import os
%matplotlib inline

In [5]:
class Config(object):
    def __init__(self, batch_size, latent_size, lr, beta1, epoch_num, alpha):
        self.batch_size = batch_size
        self.latent_size = latent_size
        self.epoch_num = epoch_num
        self.alpha = alpha
        self.lr = lr
        self.beta1 = beta1
        #self.is_training = True

In [39]:
class DCGAN(object):
    
    def __init__(self, input, sess, config):
        self.input = input
        '''self.alpha = alpha
        self.batch_size = batch_size
        self.latent_size = latent_size'''
        self.X, self.y = self.load_mnist_data()
        self.config = config
        self.sess = sess
        self.idx = 0
    def _leakyRelu(self, input, alpha, name='leakyRelu'):
        return tf.maximum(input, alpha * input, name=name)
        
    def _D_conv2d_bn(self, input, output_num, filter_size, stride, is_training=False, name=None):
        with tf.variable_scope(name) as scope:
            conv = tf.contrib.layers.conv2d(input, output_num, filter_size, stride, 
                                            'SAME', activation_fn=None, biases_initializer=None) 
            bn = tf.contrib.layers.batch_norm(inputs=conv, is_training=is_training, activation_fn=None)
            return self._leakyRelu(bn, self.config.alpha)
        
    def _G_conv_transpose_bn(self, input, output_num, filter_size, stride, is_training, name=None):
        with tf.variable_scope(name) as scope:
            conv_transpose = tf.contrib.layers.conv2d_transpose(
                input, output_num, filter_size, stride, 'SAME', 
                activation_fn=None, biases_initializer=None, scope=name)
            bn = tf.contrib.layers.batch_norm(inputs=conv_transpose,
                                              is_training=is_training, 
                                              activation_fn=None, 
                                              scope=name)
            return tf.nn.relu(bn)
        
    def discriminator(self, input, y=None, reuse=False, is_training=False):
        # LeNet-like discriminator
        with tf.variable_scope("discriminator") as scope:
            if reuse:
                scope.reuse_variables()
            d_1 = self._D_conv2d_bn(input, 8, 3, 1, is_training, name="D_conv1") # => [None, 14, 14, 8]
            d_2 = self._D_conv2d_bn(d_1, 16, 3, 2, is_training, name="D_conv2")  # => [None, 7, 7, 16]
            d_3 = self._D_conv2d_bn(d_2, 32, 3, 2, is_training, name="D_conv3")  # => [None, 7, 7, 16]
            d_4 = self._D_conv2d_bn(d_3, 64, 3, 1, is_training, name="D_conv4")  # => [None, 7, 7, 16]
            d_5 = tf.contrib.layers.flatten(d_4)
            d_6 = tf.contrib.layers.fully_connected(d_5, num_outputs=1, activation_fn=None, scope="D_linear")
            return tf.nn.sigmoid(d_6, name="D_output"), d_6
        
    def generator(self, input, reuse=False, is_training=False):
        with tf.variable_scope("generator") as scope:
            if reuse:
                scope.reuse_variable()
            g_0 = tf.cast(
                tf.contrib.layers.fully_connected(input, 7 * 7 * 128, 
                                                  activation_fn=None, 
                                                  scope="G_linear"), 
                dtype=tf.float32)
            # => [None, 4 * 4 * 1024]
            g_1 = tf.cast(tf.reshape(g_0, shape=[-1, 7, 7, 128]), dtype=tf.float32)
            g_2 = self._G_conv_transpose_bn(g_1, 64, 3, 2, is_training=is_training, name="G_conv_trans_1")
            g_3 = self._G_conv_transpose_bn(g_2, 32, 3, 1, is_training=is_training, name="G_conv_trans_2")
            g_4 = self._G_conv_transpose_bn(g_3, 16, 3, 2, is_training=is_training, name="G_conv_trans_3")
            g_5 = self._G_conv_transpose_bn(g_4, 1, 3, 1, is_training=is_training, name="G_conv_trans_4")
            return tf.nn.tanh(g_5)
            
    def build(self, config):
        self.is_training = tf.placeholder(bool, name="is_training")
        self.noise = tf.placeholder(shape=(config.batch_size, config.latent_size),
                                    dtype=tf.float32, name="noise")
        self.real_image = tf.placeholder(shape=(config.batch_size, 28, 28, 1), 
                                         dtype=tf.float32, name="real_image")
        
        self.fake_image = self.generator(self.noise, is_training=self.is_training)
        self._D_fake, self.fake_logits = self.discriminator(self.fake_image, 
                                                            y=None,
                                                            is_training=self.is_training, 
                                                            reuse=False)
        self._D_real, self.real_logits = self.discriminator(self.real_image,
                                                            y=None, 
                                                            is_training=self.is_training, 
                                                            reuse=True)
        
        
        self.G_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=self.fake_logits, 
                                                    labels=tf.ones_like(self._D_fake)))
        self.D_loss_fake = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=self.fake_logits, 
                                                    labels=tf.zeros_like(self._D_fake)))
        
        self.D_loss_real = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=self.real_logits, 
                                                    labels=tf.ones_like(self._D_real)))
        self.D_loss = self.D_loss_real + self.D_loss_fake
        
        self.G_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope="generator") 
        self.D_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope="discriminator")
        self.updata_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        
        self.G_trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="generator")
        self.D_trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="discriminator")
        self.trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        
    def train(self, config):
        with tf.control_dependencies(self.D_update_ops):
            D_opt = tf.train.AdamOptimizer(learning_rate=config.lr, beta1=config.beta1) \
                            .minimize(self.D_loss, var_list=self.D_trainable_vars)
        with tf.control_dependencies(self.G_update_ops):
            G_opt = tf.train.AdamOptimizer(learning_rate=config.lr, beta1=config.beta1) \
                            .minimize(self.G_loss, var_list=self.D_trainable_vars)
        
        noise = np.random.uniform(-1, 1, [config.batch_size, config.latent_size])
        
        sess.run(tf.global_variables_initializer())
        
        #for epoch in range(config.epoch_num):
        #    x_batch, y_batch = next_batch()
    def evaluate():
        pass
    def test():
        pass
    def load_mnist_data(self, path="mnist.npz"):
        data = np.load(path)
        X = np.concatenate((data["x_train"], data["x_test"]), axis=0)
        y = np.concatenate((data["y_test"], data["y_test"]), axis=0)
        X = np.expand_dims(X, axis=3)
        return X, y
    
    def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False):
        assert len(inputs) == len(targets)
        if shuffle:
            indices = np.arange(len(inputs))
            np.random.shuffle(indices)
        for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
            if shuffle:
                excerpt = indices[start_idx:start_idx + batch_size]
            else:
                excerpt = slice(start_idx, start_idx + batch_size)
            yield inputs[excerpt], targets[excerpt]

In [40]:
tf.reset_default_graph()

In [41]:
config = Config(batch_size=1, latent_size=100, lr=0.0002, epoch_num=1, beta1=0.5, alpha=0.2)
sess = tf.Session()
test_input = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)

test_model = DCGAN(test_input, tf.Session(), config)

In [42]:
test_model.build(config)

In [43]:
test_model.train(config)

In [44]:
XX = np.arange(10)
y  = np.arange(10)

In [89]:
for a, b in test_model.next_batch(XX, y, 2, False):
    print(a,b)

[8 9]


In [26]:
dataset = np.load("mnist.npz")

In [27]:
dataset.keys()

['x_test', 'x_train', 'y_train', 'y_test']

In [28]:
dataset['x_test'].shape

(10000, 28, 28)

In [29]:
tt = np.concatenate((dataset['x_train'], dataset['x_test']), axis=0)

In [99]:
for i,j in minibatches(XX, y, 2):
    print(i, j)

[0 1] [0 1]
[2 3] [2 3]
[4 5] [4 5]
[6 7] [6 7]
[8 9] [8 9]
